# Tarea 4

## Esteban Reyes Saldaña

### Modelos de Lenguaje Estadísticos

## 2 El Ahorcado

Diseñe una función que sea capaz de encontrar los caracteres faltantes de una palabra. Para ello proponga una adaptación simple de la estrategia de corrección ortográfica propuesta por
Norvig. La función de el ahorcado debe poder tratar con hasta 4 caracteres desconocidos en palabras de longitud arbitraria. La función debe trabajar en tiempo razonable (≈ 1 minuto en una laptop).

In [ ]:
# Corpus 
target_url = 'http://norvig.com/big.txt'
from urllib.request import urlopen
 
big = urlopen(target_url).read().decode('utf-8')

In [ ]:
import re
from collections import Counter

# Codigo Norvig
def words(text): return re.findall(r'\w+', text.lower())
 
WORDS = Counter(words(big))
 
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N
 
def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)
 
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
 
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)
 
def edits1(word):
    # Quito delets porque quiero una longitud fija
    # Quito transposiciones porque el orden es correcto
    "All edits that are one edit away from `word`."    
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(replaces + inserts)
 
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
def hangman3(word3, indexes, level = 3) :
  letters    = 'abcdefghijklmnopqrstuvwxyz'
  # Creo todos los casos de dos guiones remplazando primer guión
  # por letra del abecedario y resulevo
  if level == 3 :
    posibles = [word3[:indexes[0]] + l + word3[indexes[0]+ 1:] for l in letters]
  if level == 4 :
    posibles = [word3[:indexes[1]] + l + word3[indexes[1]+ 1:] for l in letters]

  # Busco corrección
  c = [correction(w) for w in posibles]
  # Me quedo con las que no tengan guiones
  c = [w for w in c if w.find('_') == -1]
  # Si no encuentro palabra, regreso
  if len(c) == 0 :
    return word3
  else :
    # Regreso palabras
    return c

In [ ]:
def choose_word(c) :
  return max(c, key=P)

In [ ]:
def hangman(word) :
  letters    = 'abcdefghijklmnopqrstuvwxyz'
  # Busco posiciones de guion
  indexes = [x.start() for x in re.finditer('_', word)]

  # Si solo me faltan dos letras, ya lo puedo hacer
  if len(indexes) <= 2 :
    return correction(word)
  elif  len(indexes) == 3 :
    # Paso caso de tres a un caso de dos guiones
    c = hangman3(word, indexes)
    # Si no encuentro 
    # Elijo de la de mayor probabilidad
    return choose_word(c)
  elif len(indexes) == 4 :
    for letter in letters :
      # Lo llevo al caso de tres que resolví previamente
      word3 = word[:indexes[0]] + letter + word[indexes[0]+1: ]
      c     = hangman3(word3, indexes, level = 4)
      # Muestra primera coincidencia
      if len(c) > 0 :
        return choose_word(c)
        break
  else :
    print('Only four unknown letters allow.')
    return 


In [ ]:
# 1
hangman('pe_p_e')

'people'

In [ ]:
# 2
hangman('phi__sop_y')

'philosophy'

In [ ]:
# Midiendo tiempo
tic = time.time()
hangman('phi__sop_y')
print('time: ', time.time() - tic, 'seconds.')

time:  4.3547046184539795 seconds.


In [ ]:
# 3
hangman('si_nif_c_nc_')

'significance'

In [ ]:
# Midiendo tiempo
tic = time.time()
hangman('si_nif_c_nc_')
print('time: ', time.time() - tic, 'seconds.')

time:  42.21551513671875 seconds.


In [ ]:
words = ['h_ll_'   , 'c__t_y', 's_p__', 'com____r', 'la_g_a_e', 
         'la_g_a__', 'n__l'  , '_or_d', 'n_t__a_' , 'e__', '_o__']
i = 1
for w in words: 
  z = hangman(w)
  print(f"{i} : {w:<8s} [{len(w)}] #=> {z:8s} [{len(z)}]")
  i += 1

1 : h_ll_    [5] #=> hills   [5]
2 : c__t_y   [6] #=> costly  [6]
3 : s_p__    [5] #=> supra   [5]
4 : com____r [8] #=> commoner[8]
5 : la_g_a_e [8] #=> language[8]
6 : la_g_a__ [8] #=> language[8]
7 : n__l     [4] #=> nail    [4]
8 : _or_d    [5] #=> world   [5]
9 : n_t__a_  [7] #=> natural [7]
10 : e__      [3] #=> end     [3]
11 : _o__     [4] #=> more    [4]


## 3 Modelos del Lenguaje

## 3.1 Construcción de Modelo

1. Construya un corpus para entrenar un modelo de lenguaje. Para ello use la parte de libros gutenberg del archivo big.txt, y combinelo con los datos de wikitex-2 o wikitex-103, según le convenga en tiempo y hardware. Defina su vocabulario y enmascare con \<unk> toda palabra que no esté en su vocabulari (los wikitex ya lo tienen, pero tal vez necesite acotarlo más). Valore si conviene usar el tokenizador de oraciones de nltk, o usar el texto como una oración gigante (mencioné cuál utilizó y por qué). Para este ejercicio podría no ser necesario denir tokens especiales \<s> y \</s> dependiendo de si tokenizo oraciones.
Limpie el texto dejando solo palabras en minúscula, quitando puntuación, números, etc.

**Nota**. Utilicé un tokenizador por oraciones dado que de esa forma se definió en el libro de Dan y entonces sirvió como guía para desarrolar el modelo del lenguaje paso a paso.

Limpié dichas oraciones con una expresión regular, dado que el texto tenía muchos simbolos raros inlcuso de otros idiomas.

In [ ]:
# Librerías
import os
import re
import math
import nltk 
import numpy as np

In [ ]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [ ]:
def get_texts_from_file (path_corpus, path_truth) :
  tr_txt = []
  tr_y   = []

  with open (path_corpus, "r") as f_corpus, open(path_truth, "r", encoding='utf-8') as f_truth:
    for twitt in f_corpus:
      twitt = twitt.replace('\n', ' ')
      if twitt == ' ' or twitt == '  ':
        continue
      else :
        tr_txt += [twitt]
    for label in f_truth:
      if label != ' \n' :
        tr_y   += [label]
  

  return tr_txt, tr_y

In [ ]:
dir = '/content/drive/MyDrive/CIMAT/Segundo Semestre/Lenguaje Natural/T04/wikitext-2/'

In [ ]:
tr_txt, tr_y = get_texts_from_file(dir + 'wiki.train.tokens', dir + 'wiki.test.tokens')

### Funciones para Obtener tokens

In [ ]:
# Obtengo oraciones
def split_by_punctuation (tr_txt) :
  sentences = []
  for doc in tr_txt :
    sentences += doc. split('.')
  return sentences

In [ ]:
# Tokenizo oraciones
def get_tokens_in_txt(my_txt) :
  # Expresión para quitar caracteres
  reg_exp = r"<?/?[A-Z|a-z]+>?"

  tokens_txt = []
  for doc in my_txt :
    # coniverto texto a minúsculas y limpio
    tmp = re.findall(reg_exp, doc.lower())
    tmp = ' '.join(tmp)
    # tokenizo
    tmp = tknzr.tokenize(tmp)
    # Ingoro saltos de línea
    if len(tmp) > 0:
      # Agreso caracteres de inicio y fin
      tokens_txt += [tmp]
  return tokens_txt

In [ ]:
tr_txt = split_by_punctuation(tr_txt)
tr_y   = split_by_punctuation(tr_y)

In [ ]:
big_split = big.split('\n')
big_txt   = get_tokens_in_txt(big_split)
wiki_txt  = get_tokens_in_txt(tr_txt)

In [ ]:
wiki_txt[0]

['valkyria', 'chronicles', 'iii']

In [ ]:
print('sentences in big.txt  :', len(big_txt))
print('sentences in wiki.txt :', len(wiki_txt))

sentences in big.txt  : 103417
sentences in wiki.txt : 90157


In [ ]:
# Concateno txt
clean_txt = big_txt + wiki_txt
len(clean_txt)

193574

### Creo corpus

In [ ]:
# Creo corpus
corpus_palabras = []

for doc in big_txt :
  corpus_palabras += doc

for doc in wiki_txt :
  corpus_palabras += doc

fdist = nltk.FreqDist(corpus_palabras)

In [ ]:
print('Total words :', len(corpus_palabras))
print('Vocabulary  :', len(fdist))

Total words : 2800330
Vocabulary  : 41811


In [ ]:
# Creo diccionario de frecuencias
fdist = nltk.FreqDist(corpus_palabras)

In [ ]:
# ordeno diccionario
def sortfreqDict(freqdict):
  aux = [(freqdict[key], key) for key in freqdict]
  aux.sort(key = lambda elem: (elem[0], elem[1]), reverse = True)
  return aux

In [ ]:
# Recorto vocabulario a 5000 más frecuentes contanto tokens especiales
V = sortfreqDict(fdist)
# 5000 (incluyendo <unk>)
V = V[:5000]

In [ ]:
# Creo diccionario de vocabulario
dict_indices = dict()
count = 0

for wigth, word in V:
  dict_indices[word] = count
  count += 1

In [ ]:
# Enmascaro oraciones con <unk> si no está en mi diccionario
def mask_unk(tokens_by_sentence, dict_indices) :
  final = []
  for doc in tokens_by_sentence :
    doc_ = []
    for word in doc :
      if word not in dict_indices :
        doc_.append('<unk>')
      else :
        doc_.append(word)
    final += [doc_]

  return final

In [ ]:
final_tokens = mask_unk(clean_txt, dict_indices)

In [ ]:
final_tokens[:10]

[['the',
  'project',
  'gutenberg',
  'ebook',
  'of',
  'the',
  'adventures',
  'of',
  'sherlock',
  'holmes'],
 ['by', 'sir', 'arthur', '<unk>', '<unk>'],
 ['in', 'our', 'series', 'by', 'sir', 'arthur', '<unk>', '<unk>'],
 ['copyright',
  'laws',
  'are',
  'changing',
  'all',
  'over',
  'the',
  'world',
  'be',
  'sure',
  'to',
  'check',
  'the'],
 ['copyright',
  'laws',
  'for',
  'your',
  'country',
  'before',
  '<unk>',
  'or',
  '<unk>'],
 ['this', 'or', 'any', 'other', 'project', 'gutenberg', 'ebook'],
 ['this',
  '<unk>',
  'should',
  'be',
  'the',
  'first',
  'thing',
  'seen',
  'when',
  '<unk>',
  'this',
  'project'],
 ['gutenberg',
  '<unk>',
  'please',
  'do',
  'not',
  'remove',
  'it',
  'do',
  'not',
  'change',
  'or',
  '<unk>',
  'the'],
 ['<unk>', 'without', 'written', 'permission'],
 ['please',
  'read',
  'the',
  'legal',
  'small',
  'print',
  'and',
  'other',
  'information',
  'about',
  'the']]

### Actualización de diccionario

In [ ]:
# Creo diccionario de frecuencias actualizadas
fdist = nltk.FreqDist(np.concatenate(final_tokens).ravel())

In [ ]:
fdist = nltk.FreqDist(np.concatenate(final_tokens).ravel())
# Recorto vocabulario a 5000 más frecuentes contanto tokens especiales
V = sortfreqDict(fdist)
# 5000 (incluyendo <unk>)
V = V[:5000]

In [ ]:
# Creo diccionario de vocabulario
dict_indices = dict()
count = 0

for wigth, word in V:
  dict_indices[word] = count
  count += 1

2. Entrene tres modelos de lenguaje: $ P_{unigramas}(w_1^n)$, $ P_{bigramas}(w_1^n) $, $ P_{trigramas}(w_1^n)$. Para cada uno proporcione una interfaz (función) sencilla para $ P_{n−grama}(w_1^n) $  y $ P_{n−grama}(w_1^n | w_{n-N+1}^{n-1})$. Los modelos deben tener una estrategia común para lidiar con secuencias no vistas.
Puede optar por un suavizamiento Laplace o un Good-Turing discounting.

### Unigramas

In [ ]:
len(V)

5000

In [ ]:
def unigram_probabilities(tokens_by_sentence):
    # Concateno oraciones con filtrado <unk>
    tokens = np.concatenate(tokens_by_sentence).ravel()

    # Total de palabras
    N = tokens.shape[0]

    # Creo diccionario de frecuencias actualizadas
    fdist_unigramas = nltk.FreqDist(tokens)
    
    n = len(fdist_unigramas)

    # Creo diccionario de probabilidades con suavisado de Laplace
    prob_ngramas = {word: (count + 1.0) / (N + n) for word, count in fdist_unigramas.items()}

    return prob_ngramas

In [ ]:
probabilites_unigrams =unigram_probabilities(final_tokens)

# Verifico suma
sum(probabilites_unigrams.values())

0.9999999999999991

In [ ]:
def prob_unigrams(test_sentence, probabilites_unigrams, dict_indices) :
  probability = 1.0
  
  test_sentence = test_sentence.split()
  for word in  test_sentence :
    if word not in dict_indices :
      word = '<unk>'
      probability *= probabilites_unigrams['<unk>']
    else :
      probability *= probabilites_unigrams[word]

  return probability

In [ ]:
def log_prob_unigrams(test_sentence, probabilites_unigrams, dict_indices) :
  probability = 0.0
  
  test_sentence = test_sentence.split()
  for word in  test_sentence :
    if word not in dict_indices :
      word = '<unk>'
      probability += np.log(probabilites_unigrams['<unk>'])
    else :
      probability += np.log(probabilites_unigrams[word])

  return probability

In [ ]:
np.log(probabilites_unigrams['<unk>'])

-1.9100802867464999

**Prueba**

In [ ]:
log_prob_unigrams('hello world', probabilites_unigrams, dict_indices)

-9.247776757807793

### Funciones

In [ ]:
def n_grams_probabilities(tokens_by_sentence, V, ngram = 2) :
  n      = len(V)
  n_gram = ngram - 1 
  # Creo padding para crear n-gramas con caracteres especiales
  padding = [['<s>'] * n_gram + doc + ['</s>'] * n_gram for doc in tokens_by_sentence]
  # Concateno 
  tokens = np.concatenate(padding).ravel()
  # Creo diccionario con caracteres <s> y </s>
  n_grams      = nltk.ngrams(tokens, n = ngram)
  freq_n_grams = nltk.FreqDist(n_grams)

  probabilities = dict()
  # Calculo secuencias solo si aparecen
  if ngram == 2 :
    freq_tokens  = nltk.FreqDist(tokens)

    for (first, second), count in freq_n_grams.items() :
      count_first = freq_tokens[first]

      probabilities[(second, first)] = (count + 1.0) / (count_first + n)
  if ngram == 3 :
    bigrams      = nltk.ngrams(tokens, n = ngram-1)
    freq_bigrams = nltk.FreqDist(bigrams)
    
    for (first, second, third), count in freq_n_grams.items() :
      count_fs = freq_bigrams[(first, second)]
      # Suavizado con laplace
      probabilities[(third, first, second)] = (count + 1.0) / (count_fs + n)

  return probabilities

In [ ]:
def prob_n_grams(test_sentence, probabilites_n_grams, tokens_by_sentence, V, ngram = 2) :
  n_gram = ngram - 1 
  # enmascaro oracion
  test_sentence = mask_unk([test_sentence.split()], dict_indices)
  # Concateno inicio y fin de oración
  test_sentence = ['<s>'] * n_gram + test_sentence[0] + ['</s>'] * n_gram

  # Diccionario de frecuencias unigramas
  padding = [['<s>'] * n_gram + doc + ['</s>'] * n_gram for doc in tokens_by_sentence]
  # Concateno oraciones con filtrado <unk>
  tokens = np.concatenate(padding).ravel()
  # Total de palabras
  N = tokens.shape[0]
  # Creo diccionario de frecuencias actualizadas

  n = len(V)

  probability = 1.0
  if ngram == 2 :
    fdist_unigramas = nltk.FreqDist(tokens)

    for k in range(len(test_sentence) - 1) :
      first  = test_sentence[k]
      second = test_sentence[k+1]

      if (second, first) in probabilites_n_grams :
        probability *= probabilites_n_grams[(second, first)]
      # Calculo probabilidad solo si es necesario
      else :
        freq_first   = fdist_unigramas[first]
        freq_first   = 1.0 / (freq_first + n)
        probability *= freq_first
  if ngram == 3 :
    bigrams       = nltk.ngrams(tokens, n = ngram - 1)
    fdist_bigrams = nltk.FreqDist(bigrams)

    for k in range(len(test_sentence) - 2) :
      first  = test_sentence[k]
      second = test_sentence[k+1]
      third  = test_sentence[k+2]

      if (third, first, second) in probabilites_n_grams :
        probability *= probabilites_n_grams[(third, first, second)]
      # Calculo probabilidad solo si es necesario
      else :
        count_fs = 0.0
        if (first,second) in fdist_bigrams :
          count_fs = fdist_bigrams[(first, second)]
      
        conditional_prop = 1.0 /(count_fs + n)
        probability *= conditional_prop

  return probability

In [ ]:
def log_prob_n_grams(test_sentence, probabilites_n_grams, tokens_by_sentence, V, ngram = 2) :
  n_gram = ngram - 1 
  # enmascaro oracion
  test_sentence = mask_unk([test_sentence.split()], dict_indices)
  # Concateno inicio y fin de oración
  test_sentence = ['<s>'] * n_gram + test_sentence[0] + ['</s>'] * n_gram

  # Diccionario de frecuencias unigramas
  padding = [['<s>'] * n_gram + doc + ['</s>'] * n_gram for doc in tokens_by_sentence]
  # Concateno oraciones con filtrado <unk>
  tokens = np.concatenate(padding).ravel()
  # Total de palabras
  N = tokens.shape[0]
  # Creo diccionario de frecuencias actualizadas

  n = len(V)

  probability = 0.0
  if ngram == 2 :
    fdist_unigramas = nltk.FreqDist(tokens)

    for k in range(len(test_sentence) - 1) :
      first  = test_sentence[k]
      second = test_sentence[k+1]

      if (second, first) in probabilites_n_grams :
        probability += np.log(probabilites_n_grams[(second, first)])
      # Calculo probabilidad solo si es necesario
      else :
        freq_first   = fdist_unigramas[first]
        freq_first   = 1.0 / (freq_first + n)
        probability += np.log(freq_first)
  if ngram == 3 :
    bigrams       = nltk.ngrams(tokens, n = ngram - 1)
    fdist_bigrams = nltk.FreqDist(bigrams)

    for k in range(len(test_sentence) - 2) :
      first  = test_sentence[k]
      second = test_sentence[k+1]
      third  = test_sentence[k+2]

      if (third, second, first) in probabilites_n_grams :
        probability += np.log(probabilites_n_grams[(third, first, second)])
      # Calculo probabilidad solo si es necesario
      else :
        count_fs = 0.0
        if (first,second) in fdist_bigrams :
          count_fs = fdist_bigrams[(first, second)]
      
        conditional_prop = 1.0 /(count_fs + n)
        probability += np.log(conditional_prop)

  return probability

### Bigramas

In [ ]:
probabilities_bigrams = n_grams_probabilities(final_tokens, V, ngram = 2)
# 320.7323671203257
sum(probabilities_bigrams.values())

320.7323671203257

In [ ]:
# Compruebo probabilidades condicionales fijando una palabra
conditional_prop = 0.0

for (w2, w1), count in probabilities_bigrams.items() :
  if w1 == '<unk>' :
    conditional_prop += count

conditional_prop

0.999845377991327

In [ ]:
p = prob_n_grams('the world', probabilities_bigrams, final_tokens, V, ngram = 2)
np.log(p)

-11.681909044634107

In [ ]:
log_prob_n_grams('the world', probabilities_bigrams, final_tokens, V, ngram = 2)

-11.681909044634107

### Trigramas

In [ ]:
probabilities_trigrams = n_grams_probabilities(final_tokens, V, ngram = 3)

sum(probabilities_trigrams.values())

759.6022069103449

In [ ]:
# Compruebo probabilidades condicionales fijando una palabra
conditional_prop = 0.0
for (w3, w2, w1), count in probabilities_trigrams.items() :
  if w1 == '<unk>'  and w2 == 'the':
    conditional_prop += count
conditional_prop

0.9687843921961431

In [ ]:
log_prob_n_grams('the world <unk>', probabilities_trigrams, final_tokens, V, ngram = 3)

-34.33943242363034

3. Construya un modelo interpolado con valores $\lambda $ fijos :

$$ \hat{P} (w_n | w_{n-2} w_{n-1}) = \lambda_1 P(w_n | w_{n-2} w_{n-1} ) + \lambda_2 P(w_n | w_{n-2} w_{n-1} ) + \lambda_3 P(w_n) .$$

Para ello experimente con el modelo en particiones de 80%, 10% y 10% para entrenar (train), ajuste de parámetros (val) y prueba (test) respectivamente. Muestre como bajan o suben las perplejidades en validación, finalmente pruebe una vez en test. Explore algunos
valores $ \hat{\lambda} $ y elija el mejor, i.e., [1/3, 1/3, 1/3],[.4, .4, .2],[.2, .4, .4],[5, 4, 1] y [.1, .4, .5].

In [ ]:
def get_n_gram_probability (test_sentence, probabilites_n_grams, fdist_n_grams, N, ngram) :
    probability = 1.0
    if ngram == 2 :
      for k in range(len(test_sentence) - 1) :
        first  = test_sentence[k]
        second = test_sentence[k+1]

        if (second, first) in probabilites_n_grams :
          probability *= probabilites_n_grams[(second, first)]
        # Calculo probabilidad solo si es necesario
        else :
          freq_first   = fdist_n_grams[first]
          freq_first   = 1.0 / (freq_first + N)
          probability *= freq_first
    if ngram == 3 :
      for k in range(len(test_sentence) - 2) :
        first  = test_sentence[k]
        second = test_sentence[k+1]
        third  = test_sentence[k+2]

        if (third, first, second) in probabilites_n_grams :
          probability *= probabilites_n_grams[(third, first, second)]
        # Calculo probabilidad solo si es necesario
        else :
          count_fs = 0.0
          if (first,second) in fdist_n_grams :
            count_fs = fdist_n_grams[(first, second)]
        
          conditional_prop = 1.0 /(count_fs + N)
          probability *= conditional_prop

    return probability

In [ ]:
def compute_perplexity (train_set, test_set, l) :
  # actualizo diccionario
  fdist = nltk.FreqDist(np.concatenate(train_set).ravel())
  V = sortfreqDict(fdist)
  V = V[:5000]
  # Creo diccionario de vocabulario
  dict_indices = dict()
  count = 0

  for wigth, word in V:
    dict_indices[word] = count
    count += 1

  probabilites_unigrams = unigram_probabilities(train_set)
  probabilities_bigrams = n_grams_probabilities(train_set, V, ngram = 2)
  probabilities_trigrams = n_grams_probabilities(train_set, V, ngram = 3)

  n_gram = 2
  # Enmascaro palabras desconocidas
  test_set = mask_unk(test_set, dict_indices)

  # Agrego padding solo del lado derecho
  padding  = [ doc + ['</s>'] * n_gram for doc in test_set]
  tokens   = np.concatenate(padding).ravel()
  trigrams = nltk.ngrams(tokens, n = 3)

  perplexity = 0.0

  NP = len(tokens)

  # Construyo modelos
  n_gram = 2
  padding    = [['<s>'] * n_gram + doc + ['</s>'] * n_gram for doc in train_set]
  # Concateno oraciones con filtrado <unk>
  tokens     = np.concatenate(padding).ravel()
  # Total de palabras
  N_trigrams = tokens.shape[0]
  # Frecuencias bigramas
  bigrams       = nltk.ngrams(tokens, n = 2)
  fdist_bigrams = nltk.FreqDist(bigrams)

  n_gram = 1
  padding    = [['<s>'] * n_gram + doc + ['</s>'] * n_gram for doc in train_set]
  # Concateno oraciones con filtrado <unk>
  tokens     = np.concatenate(padding).ravel()
  # Total de palabras
  N_bigrams = tokens.shape[0]
  # Frecuencias unigramas
  fdist_unigramas = nltk.FreqDist(tokens)

  perplexity = 0.0

  for (first, second, third) in trigrams :
    tmp = 0.0
    tmp += l[0] * get_n_gram_probability ((first, second, third), probabilities_trigrams, fdist_bigrams, N = N_trigrams, ngram = 3)
    tmp += l[1] * get_n_gram_probability ((first, second), probabilities_bigrams, fdist_unigramas, N = N_bigrams, ngram = 2)
    tmp += l[2] * prob_unigrams(first, probabilites_unigrams, dict_indices)

    perplexity += np.log(tmp)

    
  
  perplexity = - 1.0 / NP * perplexity
  return perplexity

In [ ]:
# Creo clase para guardar probabilidades
class language_model :
  def __init__(self, train_set, test_set, val_set = None, vocabulary_size = 5000):
    fdist = nltk.FreqDist(np.concatenate(train_set).ravel())
    V = sortfreqDict(fdist)
    V = V[:vocabulary_size]
    # Creo diccionario de vocabulario
    dict_indices = dict()
    count = 0
    for wigth, word in V:
      dict_indices[word] = count
      count += 1

    self.train_set = mask_unk(train_set, dict_indices)
    self.test_set = mask_unk(test_set, dict_indices)
    
    # Actualizo diccionario
    fdist = nltk.FreqDist(np.concatenate(self.train_set).ravel())
    V = sortfreqDict(fdist)
    self.V = V
    # Creo diccionario de vocabulario
    self.dict_indices = dict()
    count = 0
    for wigth, word in self.V:
      self.dict_indices[word] = count
      count += 1

    self.probabilites_unigrams  = unigram_probabilities(self.train_set)
    self.probabilities_bigrams  = n_grams_probabilities(self.train_set, V, ngram = 2)
    self.probabilities_trigrams = n_grams_probabilities(self.train_set, V, ngram = 3)

    # Preparo modelo para evaluación
    n_gram = 2
    padding    = [['<s>'] * n_gram + doc + ['</s>'] * n_gram for doc in self.train_set]
    # Concateno oraciones con filtrado <unk>
    tokens     = np.concatenate(padding).ravel()
    # Total de palabras
    self.N_trigrams = tokens.shape[0]
    # Frecuencias bigramas
    bigrams       = nltk.ngrams(tokens, n = 2)
    self.fdist_bigrams = nltk.FreqDist(bigrams)

    n_gram = 1
    padding    = [['<s>'] * n_gram + doc + ['</s>'] * n_gram for doc in self.train_set]
    # Concateno oraciones con filtrado <unk>
    tokens     = np.concatenate(padding).ravel()
    # Total de palabras
    self.N_bigrams = tokens.shape[0]
    # Frecuencias unigramas
    self.fdist_unigramas = nltk.FreqDist(tokens)
  
  def compute_perplexity(self, l) :
    n_gram = 2
    # Enmascaro palabras desconocidas
    test_set = mask_unk(self.test_set, self.dict_indices)

    # Agrego padding solo del lado derecho
    padding  = [ doc + ['</s>'] * n_gram for doc in test_set]
    tokens   = np.concatenate(padding).ravel()
    trigrams = nltk.ngrams(tokens, n = 3)

    NP = len(tokens)
    perplexity = 0.0

    for (first, second, third) in trigrams :
      tmp = 0.0
      tmp += l[0] * get_n_gram_probability ((first, second, third), self.probabilities_trigrams, self.fdist_bigrams, N = self.N_trigrams, ngram = 3)
      tmp += l[1] * get_n_gram_probability ((first, second), self.probabilities_bigrams, self.fdist_unigramas, N = self.N_bigrams, ngram = 2)
      tmp += l[2] * prob_unigrams(first, self.probabilites_unigrams, self.dict_indices)

      perplexity += np.log(tmp)

    perplexity = - 1.0 / NP * perplexity

    return perplexity

  def sentence_probability(self, sentence, l) :
    n_gram = 2
    # Enmascaro palabras desconocidas
    sentence = mask_unk(sentence, self.dict_indices)
    
    '''# Si no está en diccionario, ignoro oración
    for word in sentence[0] :
      if word == '<unk>' :
        return 0.0'''
    
    padding  = [['</s>'] + sentence[0] + ['</s>']]

    tokens   = np.concatenate(padding).ravel()
    trigrams = nltk.ngrams(tokens, n = 3)

    probability = 1.0

    for (first, second, third) in trigrams :
      tmp = 0.0
      # Compruebo valores de lambda
      if l[0] == 0.0 :
        tmp += 0.0
      else :
        tmp += l[0] * get_n_gram_probability ((first, second, third), self.probabilities_trigrams, self.fdist_bigrams, N = self.N_trigrams, ngram = 3)

      if l[1] == 0.0 :
        tmp += 0.0

      else :
        tmp *= l[1] * get_n_gram_probability ((first, second), self.probabilities_bigrams, self.fdist_unigramas, N = self.N_bigrams, ngram = 2)

      if l[2] == 0.0 :
        tmp += 0.0
      else :
        tmp += l[2] * prob_unigrams(first, self.probabilites_unigrams, self.dict_indices)

      probability *= tmp

    return probability

In [ ]:
from argparse import Namespace
args      = Namespace()
args.seed = 42

In [ ]:
from sklearn.model_selection import train_test_split
l = [[1/3, 1/3, 1/3], [0.4, 0.4, 0.2], [0.2, 0.4, 0.4], [0.5, 0.4, 0.1], [0.1, 0.4, 0.5]]

In [ ]:
X_train, X_val = train_test_split(final_tokens, train_size = 0.9, test_size = 0.1, random_state = args.seed)

mymodel = language_model(X_train, X_val)

for lam in l :
  pp = mymodel.compute_perplexity(lam)
  print('Perplexity : ', np.exp(pp), 'with ', lam)

Perplexity :  136.7051990998809 with  [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Perplexity :  158.26613453034238 with  [0.4, 0.4, 0.2]
Perplexity :  121.21907121268646 with  [0.2, 0.4, 0.4]
Perplexity :  206.30989680085594 with  [0.5, 0.4, 0.1]
Perplexity :  112.08267053566311 with  [0.1, 0.4, 0.5]


In [ ]:
X_train, X_val = train_test_split(final_tokens, train_size = 0.8, test_size = 0.1, random_state = args.seed)
mymodel = language_model(X_train, X_val)

In [ ]:
# Pruebo diferentes lambdas
for lam in l :
  pp = mymodel.compute_perplexity(lam)
  print('Perplexity : ', np.exp(pp), 'with ', lam)

Perplexity :  142.329470435333 with  [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Perplexity :  165.4256088629811 with  [0.4, 0.4, 0.2]
Perplexity :  125.99737146444386 with  [0.2, 0.4, 0.4]
Perplexity :  216.64448277849914 with  [0.5, 0.4, 0.1]
Perplexity :  116.23318676055801 with  [0.1, 0.4, 0.5]


## 3.2 Corrección ortográfica y gramatical

Implemente una función spell_correction que combine el uso del generador de candidatos de Norvig con el modelo de lenguaje entrenado en el punto anterior. La idea es detectar el error ortográco, generar candidatos, pero diseñar alguna estrategia para dejar que el modelo de lenguaje elija el mejor candidato para aplicar la corrección. Sí alguna palabra de entrada no está en su vocabulario reemplácela por \<unk>.

### Norvig

In [ ]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(big))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
import itertools
def spell_correction(input_text) :
  '''
  input_text: a text with possible misspellings represented as a word list
  Returns: a list with corrected misspellings
  '''
  # Busco Candidatos
  can = []
  for word in input_text :
    can.append(candidates(word))

  # Hago todas las posibles oraciones
  tupples = itertools.product(*can)
  
  # Convierto tuplas a lista
  corrected_words = []
  for element in tupples:
    corrected_words.append(list(element))

  # Obtengo probabilidades
  p = []
  for candi in corrected_words :
    p += [mymodel.sentence_probability([candi], [0.1, 0.4, 0.5])]
  
  # Me quedo con la de mayor probabilidad
  max = 0
  id = 0
  for i in range(len(p)) :
    if max < p[i] :
      max = p[i]
      id = i

  return corrected_words[id]

In [ ]:
def grammar_candidates(input_txt) :
  correct = []
  for word in input_txt :
    correct += [correction(word)]
  
  return correct

In [ ]:
example = ['my', 'countr' , 'is', 'biig']
spell_correction(example)

['my', 'country', 'is', 'big']

In [ ]:
grammar_candidates(example)

['my', 'count', 'is', 'big']

In [ ]:
target_txt = [['i', 'hav', 'a', 'ham'],
              ['my', 'countr' , 'is', 'biig'],
              ['i', 'want', 't0o', 'eat'],
              ['the', 'science', '0f', 'computr']]

In [ ]:
for t_txt in target_txt :
  print('Original: ', t_txt)
  print('Norvig  : ',grammar_candidates(t_txt))
  print('Modelo  : ',spell_correction(t_txt))

Original:  ['i', 'hav', 'a', 'ham']
Norvig  :  ['i', 'had', 'a', 'ham']
Modelo  :  ['i', 'ham', 'a', 'ham']
Original:  ['my', 'countr', 'is', 'biig']
Norvig  :  ['my', 'count', 'is', 'big']
Modelo  :  ['my', 'country', 'is', 'big']
Original:  ['i', 'want', 't0o', 'eat']
Norvig  :  ['i', 'want', 'to', 'eat']
Modelo  :  ['i', 'want', 'to', 'eat']
Original:  ['the', 'science', '0f', 'computr']
Norvig  :  ['the', 'science', 'of', 'computer']
Modelo  :  ['the', 'science', 'of', 'computer']


Explica en que consiste tu estrategia; máximo dos párrafos. Escoge algunos ejemplos que ilustren el comportamiento de su estrategia. Abajo se proporcionan algunas sugerencias de pruebas simples y su salida esperada, aunque puede usar otras si así lo desea.

**Nota.** La idea es generar los candidatos con el función de Norvig, luego calcular todas las oraciones posibles con los candidatos y luego calcular las probabilidades de acuerdo al modelo realizado y devolver el más probable.

## 3.3 Correción Gramatical

Para el escenario en el que no hay errores ortográficos, proponga una estrategia simple para sugerir corrección gramatical. Discuta en no más de dos párrafos las ventajas/desventajas de su estrategia. Escoja 3 ejemplos para ilustrar dónde su estrategia funciona y 3 en los que no. 

In [ ]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    return set(deletes + replaces)


### Ejemplos Positivos

In [ ]:
example = ['the', 'scienceses', 'of', 'computerses']

print('Original: ', example)
print('Norvig  : ',grammar_candidates(example))
print('Modelo  : ',spell_correction(example))

Original:  ['the', 'scienceses', 'of', 'computerses']
Norvig  :  ['the', 'sciences', 'of', 'computers']
Modelo  :  ['the', 'sciences', 'of', 'computers']


In [ ]:
example = ['thi', 'big', 'countri']

print('Original: ', example)
print('Norvig  : ',grammar_candidates(example))
print('Modelo  : ',spell_correction(example))

Original:  ['thi', 'big', 'countri']
Norvig  :  ['the', 'big', 'country']
Modelo  :  ['the', 'big', 'country']


In [ ]:
example = ['i', 'wana', 'go', 'to', 'the', 'batroom']

print('Original: ', example)
print('Norvig  : ',grammar_candidates(example))
print('Modelo  : ',spell_correction(example))

Original:  ['i', 'wana', 'go', 'to', 'the', 'batroom']
Norvig  :  ['i', 'want', 'go', 'to', 'the', 'bedroom']
Modelo  :  ['i', 'want', 'go', 'to', 'the', 'bedroom']


### Ejemplos Negativos

In [ ]:
example = ['the', 'science', 'off', 'maths']

print('Original: ', example)
print('Norvig  : ',grammar_candidates(example))
print('Modelo  : ',spell_correction(example))

Original:  ['the', 'science', 'off', 'maths']
Norvig  :  ['the', 'science', 'off', 'baths']
Modelo  :  ['the', 'science', 'off', 'matas']


In [ ]:
example = ['were', 'iz', 'the', 'bathroom']

print('Original: ', example)
print('Norvig  : ',grammar_candidates(example))
print('Modelo  : ',spell_correction(example))

Original:  ['were', 'iz', 'the', 'bathroom']
Norvig  :  ['were', 'in', 'the', 'bathroom']
Modelo  :  ['were', 'il', 'the', 'bathroom']


In [ ]:
example = ['the', 'bij', 'montain', 'is', 'hery']

print('Original: ', example)
print('Norvig  : ',grammar_candidates(example))
print('Modelo  : ',spell_correction(example))

Original:  ['the', 'bij', 'montain', 'is', 'hery']
Norvig  :  ['the', 'big', 'contain', 'is', 'her']
Modelo  :  ['the', 'big', 'contain', 'is', 'very']


La estrategia fue eliminar de las correcciones de Norvig aquellas que no corresponden a errores gramaticales como las transposiciones y las inserciones dado que la palabra está ''completa'' pero no corresponde a errores ortográficos.

## 3.4 Auto-completado


Use su modelo de lenguaje para diseñar una función que sugiera la siguiente palabra dada una secuencia previa.

In [ ]:
def autocompletate_and_candidates(t_text, model, l, show_other_candidates = 5) :
  # Creo todas las posibles oraciones
  sentences = []
  for word in model.dict_indices :
    sentences.append(t_text + [word])

  # Obtengo probabilidades
  p = []
  id = 0
  for sentence in sentences :
    p += [[model.sentence_probability([sentence], l), id]]
    id += 1
  # Ordeno oraciones por probabilidad
  p.sort(reverse = True)

  most_freq = [sentences[p[i+1][1]] for i in range(show_other_candidates)]
  
  return sentences[p[0][1]], most_freq



Escoja 3 ejemplos para ilustrar dónde su estrategia funciona y 3 en los que no. En un párrafo explique brevemente porque falló.

### Ejemplos Positivos

In [ ]:
s = ['i', 'would', 'like', 'to', 'visit', 'new']

top, top_five = autocompletate_and_candidates(s, mymodel,[0.1, 0.4, 0.5])
print('Top option     : ',top)
for t in top_five :
  print('Top candidates : ',t)

Top option     :  ['i', 'would', 'like', 'to', 'visit', 'new', 'york']
Top candidates :  ['i', 'would', 'like', 'to', 'visit', 'new', 'england']
Top candidates :  ['i', 'would', 'like', 'to', 'visit', 'new', 'zealand']
Top candidates :  ['i', 'would', 'like', 'to', 'visit', 'new', 'jersey']
Top candidates :  ['i', 'would', 'like', 'to', 'visit', 'new', 'orleans']
Top candidates :  ['i', 'would', 'like', 'to', 'visit', 'new', 'bone']


In [ ]:
s = ['i', 'am', 'drinking']

top, top_five = autocompletate_and_candidates(s, mymodel,[0.1, 0.4, 0.5])
print('Top option     : ',top)
for t in top_five :
  print('Top candidates : ',t)

Top option     :  ['i', 'am', 'drinking', 'water']
Top candidates :  ['i', 'am', 'drinking', 'he']
Top candidates :  ['i', 'am', 'drinking', 'of']
Top candidates :  ['i', 'am', 'drinking', 'and']
Top candidates :  ['i', 'am', 'drinking', 'from']
Top candidates :  ['i', 'am', 'drinking', 'a']


In [ ]:
s = ['i','have', 'to', 'work', 'at', 'the']

top, top_five = autocompletate_and_candidates(s, mymodel,[0.1, 0.4, 0.5])
print('Top option     : ',top)
for t in top_five :
  print('Top candidates : ',t)

Top option     :  ['i', 'have', 'to', 'work', 'at', 'the', 'same']
Top candidates :  ['i', 'have', 'to', 'work', 'at', 'the', 'time']
Top candidates :  ['i', 'have', 'to', 'work', 'at', 'the', 'first']
Top candidates :  ['i', 'have', 'to', 'work', 'at', 'the', 'end']
Top candidates :  ['i', 'have', 'to', 'work', 'at', 'the', 'other']
Top candidates :  ['i', 'have', 'to', 'work', 'at', 'the', 'french']


### Ejemplos negativos

In [ ]:
s = ['the', 'mountain', 'is', 'full', 'of']

top, top_five = autocompletate_and_candidates(s, mymodel,[0.1, 0.4, 0.5])
print('Top option     : ',top)
for t in top_five :
  print('Top candidates : ',t)

Top option     :  ['the', 'mountain', 'is', 'full', 'of', 'the']
Top candidates :  ['the', 'mountain', 'is', 'full', 'of', 'a']
Top candidates :  ['the', 'mountain', 'is', 'full', 'of', 'an']
Top candidates :  ['the', 'mountain', 'is', 'full', 'of', 'that']
Top candidates :  ['the', 'mountain', 'is', 'full', 'of', 'life']
Top candidates :  ['the', 'mountain', 'is', 'full', 'of', 'these']


In [ ]:
s = ['the', 'big', 'lake', 'is', 'full', 'of']

top, top_five = autocompletate_and_candidates(s, mymodel,[0.1, 0.4, 0.5])
print('Top option     : ',top)
for t in top_five :
  print('Top candidates : ',t)

Top option     :  ['the', 'big', 'lake', 'is', 'full', 'of', 'the']
Top candidates :  ['the', 'big', 'lake', 'is', 'full', 'of', 'a']
Top candidates :  ['the', 'big', 'lake', 'is', 'full', 'of', 'an']
Top candidates :  ['the', 'big', 'lake', 'is', 'full', 'of', 'that']
Top candidates :  ['the', 'big', 'lake', 'is', 'full', 'of', 'life']
Top candidates :  ['the', 'big', 'lake', 'is', 'full', 'of', 'these']


In [ ]:
s = ['i', 'have', 'a', 'blue']

top, top_five = autocompletate_and_candidates(s, mymodel,[0.1, 0.4, 0.5])
print('Top option     : ',top)
for t in top_five :
  print('Top candidates : ',t)

Top option     :  ['i', 'have', 'a', 'blue', 'or']
Top candidates :  ['i', 'have', 'a', 'blue', 'coat']
Top candidates :  ['i', 'have', 'a', 'blue', 'moon']
Top candidates :  ['i', 'have', 'a', 'blue', 'eyes']
Top candidates :  ['i', 'have', 'a', 'blue', 'and']
Top candidates :  ['i', 'have', 'a', 'blue', 'water']


**Nota**. Los ejemplos en los que el modelo falló fueron oraciones cortas, esto podría deberse a que la probabilidades para las oraciones sean muy similares para todas las palabras del diccionario. Además, si no se le da mucho contexto a las oraciones, parece que trata de completar con conectores o preposiciones. Para el último ejemplo que no funcionó se observa que algunas de las opciones del top_five tienen más sentido que la primera, podría tatarse de la frecuencia de las stopwords.

## 4 Generación de Texto

Para esta parte reentrenará su modelo del lenguaje interpolado:

$$ \hat{P} (w_n | w_{n-2} w_{n-1}) = \lambda_1 P(w_n | w_{n-2} w_{n-1} ) + \lambda_2 P(w_n | w_{n-2} w_{n-1} ) + \lambda_3 P(w_n) .$$

realice las siguientes actividades

1. Para los tuits de agresividad use solo palabras en minúscula, quite signos de puntuación, números, etc. Agregue tokens especiales de \<s\> y \<s\> a cada ''tuit''.


In [ ]:
def get_texts_from_file (path_corpus, path_truth) :
  tr_txt = []
  tr_y   = []

  with open (path_corpus, "r") as f_corpus, open(path_truth, "r") as f_truth:
    for twitt in f_corpus:
      twitt = twitt.replace('\n', ' ')
      tr_txt += [twitt]
    for label in f_truth:
      tr_y   += [label]
  return tr_txt, tr_y

In [ ]:
def ad_star_end_caracters(tokens) :
  final_tokens = []
  for doc in tokens :
    final_tokens += [['<s>'] + doc + ['</s>']]

  return final_tokens

In [ ]:
dir = '/content/drive/MyDrive/CIMAT/Segundo Semestre/Lenguaje Natural/Práctica 3'
tr_txt, tr_y = get_texts_from_file(dir + '/mex_train.txt', dir + '/mex_train_labels.txt')

In [ ]:
clean_txt = get_tokens_in_txt(tr_txt)
final_tokens = ad_star_end_caracters(clean_txt)

In [ ]:
final_tokens[0]

['<s>',
 'lo',
 'peor',
 'de',
 'todo',
 'es',
 'que',
 'no',
 'me',
 'dan',
 'por',
 'un',
 'tiempo',
 'y',
 'luego',
 'vuelven',
 'estoy',
 'hasta',
 'la',
 'verga',
 'de',
 'estl',
 '</s>']

2. Proponga una estrategia con base en Expectation Maximization para encontrar los valores de interpolación en $\hat{P}_{agresivo} $ usando todo el dataset de agresividad. Para ello experimente con el modelo en particiones de 80%, 10% y 10% para entrenar (train), ajuste de parámetros (val) y prueba (test) respectivamente. Muestre como bajan
las perplejidades en 5 iteraciones que usted elija (de todas las que sean necesarias de acuerdo a su EM) en validación, y pruebe una vez en test. Si no logró diseñar algo basado en EM (-10pts) explore algunos valores $\hat{lambda} $ en algunas pruebas y elija el mejor, por ejemplo: $[1/3, 1/3, 1/3],[.4, .4, .2],[.2, .4, .4],[5, 4, 1]$ y $[.1, .4, .5]$.

In [ ]:
# Separo conjunto 
X_train, X_val = train_test_split(final_tokens, train_size = 0.8, test_size = 0.2, random_state = args.seed)
X_val, X_test  = train_test_split(X_val, train_size = 0.5, test_size = 0.5, random_state = args.seed)

mymodel = language_model(X_train, X_val)

In [ ]:
def expectation_maximization (model, iterations = 5) :
  ngrams = 3
  lmdas = [1.0/ngrams for i in range(ngrams)]
  # concateno tokens en conjunto de validación
  flatten_val_set = np.concatenate(model.test_set).ravel()
  # tokens en val_set
  M = flatten_val_set.shape[0]
  # Vectores de distribuciones q_m
  q = np.zeros((M, ngrams), dtype=np.float)

  for iter in range(iterations) :
    # Ciclo sobre tokens de validación
    for m in range(M) :
      sentence = flatten_val_set[:m+1]
      q[m][0] = model.sentence_probability(sentence, [lmdas[0], 0.0, 0.0])
      q[m][1] = model.sentence_probability(sentence, [0.0, lmdas[1], 0.0])
      q[m][2] = model.sentence_probability(sentence, [0.0, 0.0, lmdas[2]])
      # Normalizo vector
      q[m] = q[m] / np.linalg.norm(q[m])
    # Update lambdas
    for z in range(ngrams) :
      lmdas[z] = 1.0 / M *sum(q[:,z])
    # Observo suma de lambdas
    print('Suma lambdas: ', sum(lmdas))
    # Observo perplejidad
    print('Perplejidad : ', np.exp(model.compute_perplexity(lmdas)))
  return lmdas

In [ ]:
l = expectation_maximization(mymodel, iterations = 5)

Suma lambdas:  1.000006642496347
Perplejidad :  190.01265766450123
Suma lambdas:  0.9999999999999999
Perplejidad :  190.01526640872692
Suma lambdas:  0.9999999999999999
Perplejidad :  190.01526640872692
Suma lambdas:  0.9999999999999999
Perplejidad :  190.01526640872692
Suma lambdas:  0.9999999999999999
Perplejidad :  190.01526640872692


In [ ]:
# Valores de lambda
l

[0.0, 0.0, 0.9999999999999999]

In [ ]:
mymodel.test_set = X_test
np.exp(mymodel.compute_perplexity(l))

189.7834877912424

3. Haga una función "tuitear" basado en su modelo de lenguaje $\hat{P}_{agresivo}$ del último punto. El modelo deberá poder parar automáticamente cuando genere algún n-grama con el símbolo de terminación de tuit al final (e.g., "\</s\>"), o 50 palabras. Proponga algo para que en los últimos tokens sea más probable generar el token "\</s\>").

In [ ]:
def tuitear (pivot, model, lmda) :
  final_sentence = pivot.copy()
  for i in range(50) :
    top, top_five = autocompletate_and_candidates(pivot, model, [0.1, 0.4, 0.5])
    pivot.pop(0)
    pivot.append(top[-1])

    final_sentence += [top[-1]]
    if top[-1] == '</s>' :
      break

  return final_sentence

In [ ]:
pivot = ['<s>', '<s>', 'hola']

# Genero oración de acuerdo a modelo
tuitear(pivot, mymodel, [0.1, 0.1, 0.8])

['<s>', '<s>', 'hola', 'pinche', 'putita', '</s>']

In [ ]:
pivot = ['<s>', '<s>', 'me', 'vale']

# Genero oración de acuerdo a modelo
tuitear(pivot, mymodel, [0.1, 0.1, 0.8])

['<s>', '<s>', 'me', 'vale', 'verga', '</s>']

In [ ]:
pivot = ['<s>', '<s>', 'los', 'gays', 'son', 'muy']

# Genero oración de acuerdo a modelo
tuitear(pivot, mymodel, [0.1, 0.1, 0.8])

['<s>', '<s>', 'los', 'gays', 'son', 'muy', 'loca', '</s>']

In [ ]:
pivot = ['<s>', '<s>', 'las', 'mujeres', 'son']

# Genero oración de acuerdo a modelo
tuitear(pivot, mymodel, [0.1, 0.1, 0.8])

['<s>', '<s>', 'las', 'mujeres', 'son', 'putas', '</s>']

**Nota**. '<\<s>' se hace más probable debido a la forma en la que se formó el corpus, añadiendo este token en cada oración.

4. Use la intuición que ha ganado en esta tarea y los datos de las mañaneras
para entrenar un modelo de lenguaje AMLO. Haga una un función "dar_conferencia()". Puede entrenar su modelo con un solo string gigante de todas las intervenciones de AMLO, sin tokens "\<s>" y "\</s>". Generé un discurso de 300 palabras y detenga al modelo de forma abrupta.

In [ ]:
dir = '/content/drive/MyDrive/CIMAT/Segundo Semestre/Lenguaje Natural/Conferencias/'

In [ ]:
import glob
import io
# Creo String vacío
conferencias_str = ""
for my_file in glob.glob(dir + "./estenograficas_limpias_por_fecha/*"):
    conferencias_str += open(my_file, "r", encoding = "utf-8").read()

In [ ]:
# Tokenizo oraciones
def get_tokens_in_txt(my_txt) :
  # Expresión para quitar caracteres
  reg_exp = r'\w+'

  tokens_txt = []
  for doc in my_txt :
    # coniverto texto a minúsculas y limpio
    tmp = re.findall(reg_exp, doc.lower())
    # Ingoro aquellas oraciones de longitud menor que 5
    # Intervenciones tienen al menos cinco tokens ['presidente', 'andrés', 'manuel', 'lópez', 'obrador']
    # Ignoro oraciones que empeicen con números
    if len(tmp) > 5 and tmp[0].isnumeric() == False :
      # Agreso caracteres de inicio y fin
      tokens_txt += [tmp]
  return tokens_txt

In [ ]:
AMLO_tokens = get_tokens_in_txt(conferencias_str.split('\n'))
AMLO_tokens[2]

['como',
 'acordamos',
 'vamos',
 'a',
 'presentar',
 'un',
 'informe',
 'sobre',
 'seguridad',
 'lo',
 'que',
 'se',
 'ha',
 'venido',
 'haciendo',
 'los',
 'resultados',
 'en',
 'este',
 'tema',
 'tan',
 'complejo',
 'y',
 'tan',
 'delicado',
 'que',
 'le',
 'importa',
 'mucho',
 'mucho',
 'a',
 'los',
 'mexicanos']

In [ ]:
# Separo conjunto 
X_train, X_val = train_test_split(AMLO_tokens, train_size = 0.8, test_size = 0.2, random_state = args.seed)
X_val, X_test  = train_test_split(X_val, train_size = 0.5, test_size = 0.5, random_state = args.seed)

mymodel = language_model(X_train, X_val, vocabulary_size = 20000)

In [ ]:
l = [[1/3, 1/3, 1/3], [0.4, 0.4, 0.2], [0.2, 0.4, 0.4], [0.5, 0.4, 0.1], [0.1, 0.4, 0.5]]
for lam in l :
  pp = mymodel.compute_perplexity(lam)
  print('Perplexity : ', np.exp(pp), 'with ', lam)

Perplexity :  440.7509166973431 with  [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Perplexity :  502.9404754374155 with  [0.4, 0.4, 0.2]
Perplexity :  384.736931055309 with  [0.2, 0.4, 0.4]
Perplexity :  641.1499331336446 with  [0.5, 0.4, 0.1]
Perplexity :  352.43930812129275 with  [0.1, 0.4, 0.5]


In [ ]:
import random 
def dar_conferencia (pivot, model, lmda, size = 300) :
  final_sentence = pivot.copy()
  for i in range(size) :
    top, top_five = autocompletate_and_candidates(pivot, model, lmda)
    top = random.choice(top_five)
    # Actualizo secuancia
    pivot.pop(0)
    pivot.append(top[-1])

    final_sentence += [top[-1]]
  return ' '.join(final_sentence)

In [ ]:
#Prueba de longitud corta

pivot = ['buenos','dias']

dar_conferencia(pivot, mymodel, [0.1, 0.4, 0.5], size = 100)

'buenos dias con eso ya se va reduciendo anualmente vamos adelante a pemex en veracruz lamentablemente se tiene información de <unk> y de acuerdo con los trabajadores petroleros a los <unk> y no no sé bien <unk> a <unk> y de acuerdo sobre los trabajadores electricistas a trabajadores sindicalizados pidan refugio y ya estamos viendo todo tipo y de acuerdo también el presidente madero el país ahora que hay una situación difícil no vamos aquí a <unk> y de salud hospitales covid en todos los estados nacionales extranjeros deberán presentarnos el bienestar y se está trabajando todos estos programas precisamente zedillo y'

In [ ]:
pivot = ['buenos','dias']

dar_conferencia(pivot, mymodel, [0.1, 0.4, 0.5])

'buenos dias si lo vemos aquí no había antes y eso fue <unk> a <unk> de la corrupción política migratoria el tema migratorio se puede probar técnica y que <unk> con <unk> de los ciudadanos <unk> que es <unk> y no no <unk> de las autoridades sanitarias que tiene <unk> que tenemos aquí también en general de seguridad ciudadana <unk> y de los medios nacionales e hijos no no hay impunidad y todo mundo se tiene <unk> a <unk> que tenemos todos los trabajadores mineros como el <unk> a los ciudadanos que <unk> que se tiene pensado para <unk> a la población afectada resaltando los <unk> a los mexicanos allá no todos los días aquí mismo se tiene <unk> y no sé yo soy partidario en eso es importante señalar es así la epidemia gracias gracias a <unk> y ya hay <unk> y ya hay <unk> no sólo se ha dicho nada sobre eso estamos aquí no sé nada <unk> nada de corrupción en el caso específico la gente que se tiene pensado que se va <unk> de las <unk> de que es importante también esto se puede probar es cier

In [ ]:
pivot = ['hoy','vamos', 'a', 'hablar', 'de']

dar_conferencia(pivot, mymodel, [0.1, 0.4, 0.5])

'hoy vamos a hablar de <unk> a los ciudadanos que se va comportando de <unk> y ya lo de <unk> a ver con las autoridades federales y de los <unk> de las <unk> a méxico y de <unk> no no hay un cambio profundo en eso sí calienta pues entonces eso sí no tienen para nosotros <unk> que tenemos ya no está bien gracias gracias por <unk> que tenemos nosotros que ver si hay alguna situación específica que sí sí se tiene contemplado el que tiene menos del 50 metros construcción total 663 mil <unk> y se puede decir más detalles que pudiera salir pronto para poder atender a la ley laboral de estados fronterizos el caso del país presidente amlo declaratoria de méxico aquí ya lo <unk> de que <unk> que tiene menos <unk> que tenemos una parte operativa que ver eso ya cambió porque es el país también para no violencia la secretaría del gobierno mexicano del petróleo que <unk> que tiene usted de eso es que es importante es el país <unk> todo todo completamente en eso es importante <unk> a ver aquí no hab